# IMPORT ED AUTENTICAZIONE

In [7]:
import os
import json
import tweepy
import math
import pprint as pp

In [8]:
#Twitter API credentials
api_key = "KLRpHp13jm62W7DK0QT4eX3d7"
api_secret = "30S9ZtJFYS6MLFUwdW7DVUgU6a0AXJ9ZqLw7Sey0Jr4ZU95XcU"
access_token = "1455882501130493955-uGFA6LQQvtxjQC9b7UoD7DCnjZhscy"
access_secret = "gEDVqti3J4gt0NzqM8PFdOYHrPQFkppwVS0redI8JzNbd"
bearer_token = "AAAAAAAAAAAAAAAAAAAAAFOvVwEAAAAAVu7OI2bxAjD1D2mDIcAIxVpGZfc%3DKQuU9n4e86fE6Ek7DROqwAyqTO9ZkyEMYnJBRKkA37rivDgXWP"

auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
if(api.verify_credentials):
    print ('Authentication completed successfully!')

client = tweepy.Client(
    consumer_key=api_key,
    consumer_secret=api_secret,
    access_token=access_token,
    access_token_secret=access_secret,
    bearer_token=bearer_token,
    wait_on_rate_limit=True,
    return_type=dict
)


Authentication completed successfully!


AttributeError: module 'tweepy' has no attribute 'Client'

# GESTIONE JSON

In [ ]:
data_folder = "data_v2"

def serialize_json(folder, filename, data):
    if not os.path.exists(folder):
        os.makedirs(folder, exist_ok=True)
    with open(f"{folder}/{filename}", 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
        f.close()
    print(f"Data serialized to path: {folder}/{filename}")

def read_json(path):
    if os.path.exists(path):
        with open(path, "r", encoding="utf8") as file:
            data = json.load(file)
        print(f"Data read from path: {path}")
        return data
    else:
        print(f"No data found at path: {path}")
        return {}


# FOLLOWERS

In [ ]:
def findUsers(utenti,mode):
    followers = {}
    for utente in utenti:

        page_counter = 0
        data_for_page=10
        data_total=5
        next_token = None

        response = client.get_users(usernames=utente)

        for user in response['data']:

            print(f"Processing user with id {user['id']} and username {user['username']}")
            followers_of_user = []

            while (next_token and page_counter<= math.ceil(data_total/data_for_page) or len(followers_of_user)<=0):

                if next_token:
                    if mode==0:
                        response = client.get_users_followers(
                            id=user['id'], user_fields=['name', 'description', 'location','followers_count','friends_count','protected'],
                            max_results=data_for_page, pagination_token=next_token
                        )
                    else:
                        response = client.get_users_following(
                            id=user['id'], user_fields=['name', 'description', 'location','followers_count','friends_count','protected'],
                            max_results=data_for_page, pagination_token=next_token
                        )
                else:
                    if mode==0:
                        response = client.get_users_followers(
                            id=user['id'], user_fields=['name', 'description', 'location','followers_count','friends_count','protected'],
                            max_results=data_for_page
                        )
                    else:
                        response = client.get_users_following(
                            id=user['id'], user_fields=['name', 'description', 'location','followers_count','friends_count','protected'],
                            max_results=data_for_page
                        )
                next_token = response['meta']['next_token'] if 'next_token' in response['meta'] else None

                for data in response['data']:
                    if(len(followers_of_user)<data_total and data['followers_count']<1000 and data['friends_count']<1000 and data['protected']==False):
                        followers_of_user.append(data)

                page_counter=page_counter+1

            followers[user] = followers_of_user
            #print(f"Found {len(followers[user])} followers for user {user}")

    serialize_json(data_folder, "users_followers.json", followers)
    return followers


l1_followers=findUsers(["mizzaro","Miccighel_"],0) #Followers di mizzaro e Miccighel
l1_following=findUsers(["mizzaro","Miccighel_"],1) #Following di mizzaro e Miccighel

l2a_followers_mizzaro=findUsers(l1_followers["mizzaro"],0) #Followers dei followers di mizzaro
l2a_following_mizzaro=findUsers(l1_followers["mizzaro"],1) #Following dei followers di mizzaro

l2a_followers_miccighel=findUsers(l1_followers["Miccighel_"],0) #Followers dei followers di Miccighel
l2a_following_miccighel=findUsers(l1_followers["Miccighel_"],1) #Following dei followers di Miccighel

l2b_followers_mizzaro=findUsers(l1_following["mizzaro"],0) #Followers dei following di mizzaro
l2b_following_mizzaro=findUsers(l1_following["mizzaro"],1) #Following dei following di mizzaro

l2b_followers_miccighel=findUsers(l1_following["Miccighel_"],0) #Followers dei following di Miccighel
l2b_following_miccighel=findUsers(l1_following["Miccighel_"],1) #Following dei following di Miccighel

In [ ]:
followers = read_json(f"{data_folder}/avengers_followers.json")
# (Pretty) print the first three followers of Iron Man
import pprint
pp = pprint.PrettyPrinter()
pp.pprint(followers['CaptainAmerica'][:3])

# ESEMPI

# 1 - Scaricate i dettagli del profilo di Capitan America

In [ ]:
avenger = "CaptainAmerica"
profile_data = client.get_users(
    usernames=avenger,
    user_fields=['name', 'description', 'location', 'public_metrics']
    )
serialize_json(data_folder, "1_captain_america_profile_data.json", profile_data)
pp.pprint(profile_data)


# 2 - Scaricate gli ultimi 15 tweet di Captain America

In [ ]:
avenger_id = 701615052
page_counter = 0
data_for_page=10
data_total=15
next_token = None
tweets = []

while (next_token and page_counter<= math.ceil(data_total/data_for_page) or len(tweets)<=0):
    if next_token:
        response = client.get_users_tweets(
            id=avenger_id, tweet_fields=['created_at', 'lang', 'public_metrics'],
            max_results=data_for_page, pagination_token=next_token
        )
    else:
        response = client.get_users_tweets(
            id=avenger_id, tweet_fields=['created_at', 'lang', 'public_metrics'],
            max_results=data_for_page
        )
    next_token = response['meta']['next_token'] if 'next_token' in response['meta'] else None
    for data in response['data']:
        if(len(tweets)<data_total):
            tweets.append(data)
    page_counter=page_counter+1
pp.pprint(tweets)
serialize_json(data_folder, "2_captain_america_recent_tweets.json", tweets)


# 3 - Scaricate 20 tweet preferiti da Iron Man

In [ ]:
avenger = "Iron_Man"
raw_favorites = api.favorites(
    screen_name=avenger,
    tweet_mode="extended"
)
tweets = []
for raw_tweet in raw_favorites:
    json_tweet = raw_tweet._json
    tweet = {}
    tweet["id"]= json_tweet["id"]
    tweet["created_at"] = json_tweet["created_at"]
    tweet["full_text"] = json_tweet["full_text"],
    tweet["favorite_count"]= json_tweet["favorite_count"]
    tweets.append(tweet)
serialize_json(data_folder, "3_iron_man_favorite_tweets.json", tweets)
pp.pprint(tweets)


In [ ]:
import requests

response = requests.get("https://api.github.com/events")
# Print just the first event
pp.pprint(response.json()[0])
response = requests.post("http://httpbin.org/post")
response = requests.put("http://httpbin.org/put")
response = requests.delete("http://httpbin.org/delete")

In [ ]:
endpoint_url = "https://api.twitter.com/2/users/by"
headers = {
    'authorization': f"Bearer {bearer_token}"
}
params = {
    'usernames': 'CaptainAmerica',
    'user.fields': 'description,created_at,location'
}
response = requests.get(endpoint_url, headers=headers, params=params)
pp.pprint(response.json())
